In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Preparing Data

In [2]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm')
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root="../assets/data")

In [3]:
print(train_data[0].text)
print(train_data[0].label)

['it', "'s", 'movies', 'like', 'these', 'that', 'make', 'you', 'wish', 'that', 'you', 'never', 'picked', 'on', 'the', 'nerd', 'growing', 'up', 'in', 'school', '.', 'If', 'you', 'liked', 'this', 'movie', ',', 'then', 'I', 'would', 'suggest', 'you', 'watch', 'Valentine', '.', 'I', 'just', 'found', 'out', 'today', 'that', 'the', 'guy', 'who', 'played', 'Marty(Simon', ')', 'killed', 'himself', 'a', 'little', 'after', 'the', 'movie', 'was', 'released', 'which', 'is', 'a', 'shame', 'since', 'he', 'did', 'a', 'good', 'job', '.', 'I', 'wonder', 'if', 'it', "'s", 'because', 'of', 'the', 'part', 'he', 'played', 'in', 'the', 'movie', '.', 'It', 'starts', 'out', 'when', 'Carol', 'tricks', 'him', 'into', 'going', 'into', 'the', 'girls', 'restroom', 'to', 'act', 'like', 'they', 'were', 'about', 'to', 'do', 'it', '.', 'When', 'he', 'was', 'changing', 'in', 'the', 'showers', ',', 'Carols', 'popular', 'friends', 'snuck', 'into', 'the', 'bathroom', 'and', 'got', 'everything', 'ready', ',', 'camera', ','

In [4]:
train_data, valid_data = train_data.split()
len(train_data), len(valid_data), len(test_data)

(17500, 7500, 25000)

In [5]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(TEXT.vocab.itos[:10])
print(TEXT.vocab.freqs.most_common(10))

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
[('the', 203481), (',', 192431), ('.', 166011), ('and', 110001), ('a', 109928), ('of', 101023), ('to', 94106), ('is', 76470), ('in', 61621), ('I', 54616)]


In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

# Build Model

In [7]:
class Classifier(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, out_dim):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim)
        self.rnn = nn.RNN(emb_dim, hid_dim)
        self.fc = nn.Linear(hid_dim, out_dim)

    def forward(self, text):
        # text: (step, batch)
        embedded = self.emb(text)
        # hidden: (1, batch, hid_dim)
        outs, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
HID_DIM = 256
OUT_DIM = 1

classifier = Classifier(IN_DIM, EMB_DIM, HID_DIM, OUT_DIM).to(device)
count_parameters(classifier)

2592105

# Train Model

In [9]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        preds = classifier(batch.text).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            preds = classifier(batch.text).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [10]:
optimizer = optim.AdamW(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [11]:
import time
N_EPOCHS = 5
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), "models/tut1-model.pt")
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: 0.699 | Train Acc: 49.96%
	 Val. Loss: 0.702 |  Val. Acc: 49.16%
Epoch: 02 | Epoch Time: 0m 22s
	Train Loss: 0.702 | Train Acc: 49.97%
	 Val. Loss: 0.694 |  Val. Acc: 49.95%
Epoch: 03 | Epoch Time: 0m 21s
	Train Loss: 0.697 | Train Acc: 50.20%
	 Val. Loss: 0.694 |  Val. Acc: 50.53%
Epoch: 04 | Epoch Time: 0m 20s
	Train Loss: 0.697 | Train Acc: 49.06%
	 Val. Loss: 0.694 |  Val. Acc: 49.52%
Epoch: 05 | Epoch Time: 0m 21s
	Train Loss: 0.697 | Train Acc: 50.11%
	 Val. Loss: 0.693 |  Val. Acc: 49.87%
